In [1]:
# note on high redshift standardizable candles

In [1]:
%load_ext autoreload
%autoreload 2

In [43]:
#from MyUnit import NaturalUnit
import numpy as np
import sys,os
import matplotlib
import matplotlib.pyplot as plt
from scipy.special import gamma 
from scipy.integrate import quad

path = '/home/chen/Code/SN/axions_emcee_v7_LICM_38_realization/'
sys.path.insert(1, path)
import igm as ig
import data as dt
import chi2 as ch

from ag_probs import omega_plasma

In [38]:
data = dt.load_quasars("./likelihoods", "quasar.dat")
(qso_name_arr,
 qso_z_arr,
 qso_f2500_arr,
 qso_df2500_arr,
 qso_f2keV_arr,
 qso_df2keV_low_arr,
 qso_df2keV_up_arr) = data

In [76]:
# x = (ma, ga, OmL, h0, qso_gamma, qso_beta)
x = (1e-15, 1e-11, 0.7, 0.7, 0.5, 0.5)

# quasar kwargs
quasars_kwargs = {'B': 1., # [nG]
                  'mg': omega_plasma(1.6e-8), # omega_plasma(ne_IGM),
                  's': 1., # [Mpc]
                  'omega_X': 2000.,  
                  'omega_UV': 4.96, 
                  'axion_ini_frac': 0.,
                  'smoothed': False,
                  'redshift_dependent': True,
                  'method': 'simps',
                  'prob_func': 'norm_log',
                  'Nz': 101}



In [77]:
%%time
# for loop
ch.chi2_quasars(x, data=data, **quasars_kwargs)

CPU times: user 1.27 s, sys: 0 ns, total: 1.27 s
Wall time: 1.2 s


209603600.857891

In [78]:
%%time
# vectorized
ch.chi2_quasars(x, data=data, vectorize=True, **quasars_kwargs)

CPU times: user 1.16 s, sys: 0 ns, total: 1.16 s
Wall time: 1.1 s


209603600.8578905